In [1]:
from shapely.geometry import Point, LineString, MultiLineString
import geopandas as gpd
import numpy as np

import math
import bisect

In [2]:
def build_pt_trajectory(route: MultiLineString, trajectory_start_time: int, trajectory_end_time: int, time_step: int = 10, reverse: bool = False):
    # get points
    path_coordinates = []
    for single_linestring in route.geoms:
        for point_tuple in list(single_linestring.coords):
            path_coordinates.append(tuple(point_tuple))

    if reverse:
        path_coordinates.reverse()

    segment_lengths = []
    cumulative_distances = [0.0]
    current_total_distance = 0.0

    for i in range(len(path_coordinates) - 1):
        p1 = path_coordinates[i]
        p2 = path_coordinates[i+1]

        dist = math.hypot(p2[0] - p1[0], p2[1] - p1[1])
        segment_lengths.append(dist)
        current_total_distance += dist
        cumulative_distances.append(current_total_distance)

    total_trajectory_length = current_total_distance

    trajectory_total_duration = trajectory_end_time - trajectory_start_time
    velocity = total_trajectory_length / trajectory_total_duration

    times_to_check = []
    current_t_gen = trajectory_start_time
    while current_t_gen <= trajectory_end_time + 1e-9:
        times_to_check.append(current_t_gen)
        current_t_gen += time_step

    output_points = {}
    for t_query in times_to_check:
        if not (trajectory_start_time - 1e-9 <= t_query <= trajectory_end_time + 1e-9):
            continue
        time_on_trajectory = t_query - trajectory_start_time
        
        distance_to_find = velocity * time_on_trajectory
        distance_to_find = max(0.0, min(distance_to_find, total_trajectory_length))

        interpolated_point = None
        if math.isclose(distance_to_find, 0.0):
                interpolated_point = path_coordinates[0]
        elif math.isclose(distance_to_find, total_trajectory_length):
            interpolated_point = path_coordinates[-1]
        else:
            segment_insertion_idx = bisect.bisect_left(cumulative_distances, distance_to_find)
            segment_idx_in_arrays = segment_insertion_idx - 1

            if segment_idx_in_arrays < 0: segment_idx_in_arrays = 0
            if segment_idx_in_arrays >= len(segment_lengths): segment_idx_in_arrays = len(segment_lengths) - 1

            p_start_segment = path_coordinates[segment_idx_in_arrays]
            p_end_segment = path_coordinates[segment_idx_in_arrays + 1]
            length_of_this_segment = segment_lengths[segment_idx_in_arrays]

            if math.isclose(length_of_this_segment, 0.0):
                interpolated_point = p_start_segment
            else:
                dist_into_segment = distance_to_find - cumulative_distances[segment_idx_in_arrays]
                ratio = dist_into_segment / length_of_this_segment
                ratio = max(0.0, min(1.0, ratio))

                interp_x = p_start_segment[0] + ratio * (p_end_segment[0] - p_start_segment[0])
                interp_y = p_start_segment[1] + ratio * (p_end_segment[1] - p_start_segment[1])
                interpolated_point = (interp_x, interp_y)
        if interpolated_point:
                output_points[t_query] = interpolated_point
    return output_points

In [3]:
# load geojson
pt_route_filepath = 'data/sum-wp2_3-deliverable/pt/example_network/example_gtfs/example_pt_route.geojson'
pt_routes = gpd.read_file(pt_route_filepath)

# get line string
route: MultiLineString = pt_routes[pt_routes['id'] == 0]['geometry'].values[0]

In [4]:
study_start_time = 0
study_end_time = 7200
pt_headway = 120
pt_duration = 180
time_step = 10

In [5]:
def build_pt_trajectory_during_study(pt_trajectory: dict, study_start_time: int, study_end_time: int, pt_headway: int, pt_duration: int, time_step: int = 10, reverse: bool = False):
    trajectory_times_to_check = []
    # get all possible trajectory_start_time, trajectory_end_time
    for t in range(study_start_time, study_end_time, pt_headway):
        if t + pt_duration > study_end_time:
            break
        trajectory_times_to_check.append((t, t + pt_duration))

    pt_trajectories = {}
    for trajectory_start_time, trajectory_end_time in trajectory_times_to_check:
        pt_trajectories[(trajectory_start_time, trajectory_end_time)] = build_pt_trajectory(route, trajectory_start_time, trajectory_end_time, time_step, reverse)

    new_pt_trajectories = {}
    for _, nested_dict in pt_trajectories.items():
        for time_key, coordinates in nested_dict.items():
            if time_key not in new_pt_trajectories:
                new_pt_trajectories[time_key] = []
            new_pt_trajectories[time_key].append(coordinates)
    return new_pt_trajectories

In [6]:
pt_trajectories = build_pt_trajectory_during_study(route, study_start_time, study_end_time, pt_headway, pt_duration, time_step)
pt_trajectories_reverse = build_pt_trajectory_during_study(route, study_start_time, study_end_time, pt_headway, pt_duration, time_step, reverse=True)

In [7]:
pt_trajectories

{0: [(696927.5526839164, 5329616.926647293)],
 10: [(696866.5432487273, 5329666.312846756)],
 20: [(696818.8703216251, 5329728.8002404105)],
 30: [(696783.4809732959, 5329799.036471702)],
 40: [(696759.193470199, 5329873.8669474535)],
 50: [(696745.1506546512, 5329951.28802872)],
 60: [(696740.9242669137, 5330029.8621987095)],
 70: [(696746.4813929439, 5330108.351833436)],
 80: [(696762.2307033327, 5330185.4374194555)],
 90: [(696788.1634814298, 5330259.745396482)],
 100: [(696815.6944882872, 5330333.514133822)],
 110: [(696843.2254951446, 5330407.282871163)],
 120: [(696870.756502002, 5330481.051608504),
  (696927.5526839164, 5329616.926647293)],
 130: [(696896.7894100499, 5330555.354956283),
  (696866.5432487273, 5329666.312846756)],
 140: [(696922.1809729078, 5330629.887174036),
  (696818.8703216251, 5329728.8002404105)],
 150: [(696947.5725357657, 5330704.419391789),
  (696783.4809732959, 5329799.036471702)],
 160: [(696972.9640986236, 5330778.951609541),
  (696759.193470199, 53298

In [8]:
pt_trajectories_reverse

{0: [(697023.7472243394, 5330928.016045045)],
 10: [(696998.3556614815, 5330853.483827293)],
 20: [(696972.9640986236, 5330778.951609541)],
 30: [(696947.5725357657, 5330704.419391789)],
 40: [(696922.1809729078, 5330629.887174036)],
 50: [(696896.7894100499, 5330555.354956283)],
 60: [(696870.756502002, 5330481.051608504)],
 70: [(696843.2254951446, 5330407.282871163)],
 80: [(696815.6944882872, 5330333.514133822)],
 90: [(696788.1634814298, 5330259.745396482)],
 100: [(696762.2307033327, 5330185.4374194555)],
 110: [(696746.4813929439, 5330108.351833436)],
 120: [(696740.9242669137, 5330029.8621987095),
  (697023.7472243394, 5330928.016045045)],
 130: [(696745.1506546512, 5329951.28802872),
  (696998.3556614815, 5330853.483827293)],
 140: [(696759.193470199, 5329873.8669474535),
  (696972.9640986236, 5330778.951609541)],
 150: [(696783.4809732959, 5329799.036471702),
  (696947.5725357657, 5330704.419391789)],
 160: [(696818.8703216251, 5329728.8002404105),
  (696922.1809729078, 53306